In [1]:
pip install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.9 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: flask
    Found existing installation: Flask 3.1.0
    Unins

In [14]:
import numpy as np
import cv2
import open3d as o3d
import os

In [15]:
# Step I. Feature Extraction and Feature Matching
def siftFeatureExtractor(img, detector):
    """
    이미지에서 SIFT 특징점과 디스크립터를 추출합니다.
    """
    fPts, fDescs = detector.detectAndCompute(img, None)
    return fPts, fDescs

def matchSIFTFeatures(descA, descB):
    """
    BFMatcher를 사용하여 두 이미지의 디스크립터를 매칭합니다.
    """
    bruteForce = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matchedPairs = bruteForce.match(descA, descB)
    return matchedPairs

In [16]:
# Step II. Essential Matrix Estimation (RANSAC)
def estimateEssentialMat(ptsA, ptsB, intrinsicMat):
    """
    RANSAC을 이용해 두 이미지의 대응점으로부터 Essential Matrix를 추정합니다.
    """
    essentialMat, maskMat = cv2.findEssentialMat(ptsA, ptsB, intrinsicMat, method=cv2.RANSAC, threshold=0.2, maxIters=500)
    return essentialMat, maskMat

In [17]:
# Step III. Essential Matrix Decomposition
def decomposeEssentialMat(essentialMat, ptsA, ptsB, intrinsicMat):
    """
    Essential Matrix로부터 카메라의 회전과 평행 이동을 복원하고 inlier 대응점만 반환합니다.
    """
    _, rotationMat, translationVec, poseMask = cv2.recoverPose(essentialMat, ptsA, ptsB, intrinsicMat)
    inlierPtsA = ptsA[poseMask.ravel() > 0]
    inlierPtsB = ptsB[poseMask.ravel() > 0]
    return rotationMat, translationVec, inlierPtsA, inlierPtsB

In [18]:
# Step IV. Triangulation
def triangulate3DPoints(projMatA, projMatB, ptsA, ptsB):
    """
    두 뷰의 투영 행렬과 대응 2D 포인트를 이용해 3D 포인트를 삼각측량합니다.
    """
    points4D = cv2.triangulatePoints(projMatA, projMatB, ptsA.T, ptsB.T)
    points3D = cv2.convertPointsFromHomogeneous(points4D.T)
    return np.squeeze(points3D)

In [19]:
# 추가: 공통 포인트 및 마스크 생성 함수 (process_new_view에서 사용)
def findCommonCoordinates(featuresPrime, featuresOrig):
    """
    두 배열에서 동일한 좌표(공통 포인트)의 인덱스를 찾습니다.
    """
    commonIndicesOrig = []
    commonIndicesPrime = []
    for i in range(featuresOrig.shape[0]):
        foundIdx = np.where(np.all(featuresPrime == featuresOrig[i, :], axis=1))
        if foundIdx[0].size != 0:
            commonIndicesOrig.append(i)
            commonIndicesPrime.append(foundIdx[0][0])
    return np.array(commonIndicesOrig), np.array(commonIndicesPrime)

def generateMaskArrays(featuresPrime, featuresNext, commonIndices):
    """
    공통 인덱스를 마스킹하여 남은 포인트들만 추출합니다.
    """
    maskArrPrime = np.ma.array(featuresPrime, mask=False)
    maskArrPrime.mask[commonIndices] = True
    maskArrPrime = maskArrPrime.compressed().reshape(-1, 2)

    maskArrNext = np.ma.array(featuresNext, mask=False)
    maskArrNext.mask[commonIndices] = True
    maskArrNext = maskArrNext.compressed().reshape(-1, 2)

    return maskArrPrime, maskArrNext

In [38]:
# Step V. PnP를 통한 새로운 뷰의 Pose 추정
def processAdditionalView(viewIdx, imageA, imageB, ptsA, ptsB, projMatA, projMatB, intrinsicMat,
                          initial3DPoints, all3DPoints, allColors, allCameras, detector, currentViewCount, outputFolder):
    """
    추가 이미지들을 순차적으로 처리하면서 PnP로 새 뷰의 카메라 포즈를 추정하고,
    새로운 3D 포인트와 색상, 카메라 위치를 누적 업데이트합니다.
    또한, 각 뷰가 추가될 때마다 중간 결과를 ply 파일로 저장합니다.
    """
    if viewIdx == 0:
        imageList = [imageA, imageB]
        pointList = [ptsA, ptsB]
        poseList = [projMatA, projMatB]
    else:
        imageList = [imageB, imageA]
        pointList = [ptsB, ptsA]
        poseList = [projMatB, projMatA]

    # 초기 PnP 수행 (신뢰할 inlier만 남김)
    _, rotationVec, translationVec, inlier = cv2.solvePnPRansac(initial3DPoints, pointList[1], intrinsicMat, None)
    if inlier is not None:
        pointList[1] = pointList[1][inlier[:, 0]]
        initial3DPoints = initial3DPoints[inlier[:, 0]]

    # 새 뷰들을 순차적으로 처리 (각 그룹당 15개의 이미지)
    for subViewIndex in range(1, 16):
        imageIndex = viewIdx * 16 + subViewIndex
        imagePath = f"./my_data/{str(imageIndex).zfill(4)}.jpg" # imagePath = f"./base_data/{str(imageIndex).zfill(4)}.JPG"
        nextImage = cv2.imread(imagePath)

        # 이전 두 뷰의 대응점으로부터 삼각측량
        points4D = cv2.triangulatePoints(poseList[0], poseList[1], pointList[0].T, pointList[1].T)
        points3D = cv2.convertPointsFromHomogeneous(points4D.T).squeeze()

        # 새 이미지에서 특징 추출
        keypointsA_prime, descriptorsA_prime = detector.detectAndCompute(imageList[1], None)
        keypointsB, descriptorsB = detector.detectAndCompute(nextImage, None)

        matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
        matchesList = matcher.match(descriptorsA_prime, descriptorsB)
        queryIndices = [match.queryIdx for match in matchesList]
        trainIndices = [match.trainIdx for match in matchesList]

        featuresA_prime = np.float32([keypointsA_prime[i].pt for i in queryIndices])
        featuresB = np.float32([keypointsB[i].pt for i in trainIndices])

        # 공통 포인트 및 마스크 배열 생성
        commonIdxA, commonIdxB = findCommonCoordinates(featuresA_prime, pointList[1])
        maskArrPrime, maskArrNext = generateMaskArrays(featuresA_prime, featuresB, commonIdxB)

        commonPointsB = featuresB[commonIdxB]
        commonPointsA_prime = featuresA_prime[commonIdxB]
        commonPoints3D = points3D[commonIdxA]

        # PnP 수행하여 새 카메라 포즈 추정
        _, calcRotationVec, translationVec, inlier = cv2.solvePnPRansac(commonPoints3D, commonPointsB, intrinsicMat, None, reprojectionError=0.45)
        rotationMat, _ = cv2.Rodrigues(calcRotationVec)
        transformation1 = np.hstack((rotationMat, translationVec))

        if inlier is not None:
            commonPointsB = commonPointsB[inlier[:, 0]]
            commonPoints3D = commonPoints3D[inlier[:, 0]]
            commonPointsA_prime = commonPointsA_prime[inlier[:, 0]]

        projMat2 = intrinsicMat @ transformation1
        points4D_new = cv2.triangulatePoints(poseList[1], projMat2, maskArrPrime.T, maskArrNext.T)
        newPoints3D = cv2.convertPointsFromHomogeneous(points4D_new.T).squeeze()

        # 누적 3D 포인트 및 색상 업데이트
        all3DPoints = np.vstack((all3DPoints, newPoints3D))
        remainingPoints = np.array(maskArrNext, dtype=np.int32)
        colorVector = np.array([nextImage[ptCoord[1], ptCoord[0]] for ptCoord in remainingPoints])
        allColors = np.vstack((allColors, colorVector))

        # 카메라 포즈, 이미지, 대응점 업데이트 (다음 반복을 위해)
        poseList[0] = np.copy(poseList[1])
        poseList[1] = np.copy(projMat2)
        allCameras = np.vstack((allCameras, transformation1[:, 3].T))

        imageList[0] = np.copy(imageList[1])
        imageList[1] = np.copy(nextImage)
        pointList[0] = np.copy(featuresA_prime)
        pointList[1] = np.copy(featuresB)

        # 새로운 뷰 추가됨 (전체 뷰 수 증가)
        currentViewCount += 1

        # --- 중간 결과 ply 파일 저장 ---
        # 누적 포인트와 카메라 위치에 대해 스케일 보정 및 필터링 (저장을 위한 임시 처리)
        scaledPoints = all3DPoints * 200
        scaledCameras = allCameras * 200
        reshapedColors = allColors.reshape(-1, 3)
        currentOutput = np.hstack([scaledPoints, reshapedColors])
        meanCenter = np.mean(currentOutput[:, :3], axis=0)
        centeredOutput = currentOutput[:, :3] - meanCenter
        distances = np.sqrt(np.sum(centeredOutput**2, axis=1))
        validIdx = np.where(distances < np.mean(distances) - 1300)
        currentOutput = currentOutput[validIdx]
        # 카메라 위치에 빨간색 ([0, 0, 255]) 추가
        cameraColor = np.array([[0, 0, 255]])
        for i in range(1, len(scaledCameras)):
            cameraColor = np.vstack((cameraColor, [0, 0, 255]))
        scaledCameras = np.hstack((scaledCameras, cameraColor))
        currentOutput = np.vstack((currentOutput, scaledCameras))

        fileName = os.path.join(outputFolder, f"points{currentViewCount}_view.ply")
        exportPointCloud(currentOutput, fileName)
        print(f"Saved: {fileName}")

    return all3DPoints, allColors, allCameras, currentViewCount

In [39]:
# Point Cloud 저장 (Open3D 사용)
def exportPointCloud(pointData, fileName="points3D.ply"):
    """
    Open3D를 사용해 3D 포인트와 색상 정보를 ply 파일로 저장합니다.
    """
    pts = pointData[:, :3]
    clr = pointData[:, 3:6].astype(np.float64)
    if clr.max() > 1:
        clr = clr / 255.0
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pts)
    pcd.colors = o3d.utility.Vector3dVector(clr)
    o3d.io.write_point_cloud(fileName, pcd)

In [40]:
# 전체 SfM 파이프라인 실행
def runStructureFromMotionPipeline():
    # 결과 저장 폴더 생성
    outputFolder = "/content/my_data_ply_folder" # "/content/base_data_ply_folder"
    if not os.path.exists(outputFolder):
        os.makedirs(outputFolder)

    scaleFactor = 2.0  # (필요 시 이미지 크기 조정에 사용)

    # 카메라 내부 파라미터 로드
    intrinsicMat = np.loadtxt('./my_data/K.txt') # './base_data/K.txt'

    # 초기 이미지 로드 (2-view)
    imageA = cv2.imread('./my_data/0000.jpg') # './base_data/0000.JPG'
    imageB = cv2.imread('./my_data/0016.jpg') # './base_data/0016.JPG'

    # SIFT 검출기 생성
    detector = cv2.xfeatures2d.SIFT_create()

    # Step I: Feature Extraction
    keypointsA, descriptorsA = siftFeatureExtractor(imageA, detector)
    keypointsB, descriptorsB = siftFeatureExtractor(imageB, detector)

    # Step I: Feature Matching
    featureMatches = matchSIFTFeatures(descriptorsA, descriptorsB)

    MINIMUM_MATCH_COUNT = 10
    if len(featureMatches) < MINIMUM_MATCH_COUNT:
        print("Not enough matches were found - %d/%d" % (len(featureMatches), MINIMUM_MATCH_COUNT))
        return

    # 매칭된 대응점 좌표 추출
    queryIndices = [match.queryIdx for match in featureMatches]
    trainIndices = [match.trainIdx for match in featureMatches]
    ptsA = np.float32([keypointsA[i].pt for i in queryIndices])
    ptsB = np.float32([keypointsB[i].pt for i in trainIndices])

    # Step II: Essential Matrix Estimation (RANSAC)
    essentialMat, maskMat = estimateEssentialMat(ptsA, ptsB, intrinsicMat)
    maskMatches = maskMat.ravel().tolist()
    validMatches = [m for i, m in enumerate(featureMatches) if maskMatches[i] == 1]
    validPtsA = np.array([ptsA[i] for i, m in enumerate(featureMatches) if maskMatches[i] == 1])
    validPtsB = np.array([ptsB[i] for i, m in enumerate(featureMatches) if maskMatches[i] == 1])

    # (옵션) 매칭 결과 시각화
    imageMatches = cv2.drawMatches(imageA, keypointsA, imageB, keypointsB, validMatches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    # Step III: Essential Matrix Decomposition
    rotationMat, translationVec, inlierPtsA, inlierPtsB = decomposeEssentialMat(essentialMat, validPtsA, validPtsB, intrinsicMat)
    transformation0 = np.hstack((rotationMat, translationVec))

    # 첫 번째 카메라 위치 (원점)
    allCameras = np.zeros((1, 3))
    allCameras = np.vstack((allCameras, translationVec.T))

    # 첫 두 뷰의 투영 행렬 계산
    projMatA = intrinsicMat @ np.hstack((np.eye(3), np.zeros((3, 1))))
    projMatB = intrinsicMat @ transformation0

    ptsA = inlierPtsA
    ptsB = inlierPtsB

    # Step IV: Triangulation (초기 3D 포인트 복원)
    triangulatedPoints = triangulate3DPoints(projMatA, projMatB, ptsA, ptsB)
    initial3DPoints = triangulatedPoints

    # 2-view 상태 저장 (초기 상태)
    all3DPoints = np.zeros((1, 3))
    allColors = np.zeros((1, 3))

    scaledPoints = all3DPoints * 200
    scaledCameras = allCameras * 200
    reshapedColors = allColors.reshape(-1, 3)
    finalOutput = np.hstack([scaledPoints, reshapedColors])
    meanCenter = np.mean(finalOutput[:, :3], axis=0)
    centeredOutput = finalOutput[:, :3] - meanCenter
    distances = np.sqrt(np.sum(centeredOutput**2, axis=1))
    validIdx = np.where(distances < np.mean(distances) - 1300)
    finalOutput = finalOutput[validIdx]
    cameraColor = np.array([[0, 0, 255]])
    for i in range(1, len(scaledCameras)):
        cameraColor = np.vstack((cameraColor, [0, 0, 255]))
    scaledCameras = np.hstack((scaledCameras, cameraColor))
    finalOutput = np.vstack((finalOutput, scaledCameras))

    initialFileName = os.path.join(outputFolder, "points2_view.ply")
    exportPointCloud(finalOutput, initialFileName)
    print(f"Saved: {initialFileName}")

    # 현재 뷰 카운트 (2-view 상태 저장 후)
    currentViewCount = 2

    # Step V: PnP를 통한 새로운 뷰의 Pose 추정 (총 30개 뷰 추가하여 32-view까지 진행)
    for viewIdx in range(2):
        all3DPoints, allColors, allCameras, currentViewCount = processAdditionalView(
            viewIdx, imageA, imageB, ptsA, ptsB, projMatA, projMatB, intrinsicMat, initial3DPoints,
            all3DPoints, allColors, allCameras, detector, currentViewCount, outputFolder)

    # 최종적으로 32-view reconstruction 결과는 각 단계 저장 파일에 포함됨.
    print("SfM pipeline processing complete.")

In [41]:
if __name__ == "__main__":
    runStructureFromMotionPipeline()

Saved: /content/my_data_ply_folder2/points2_view.ply
Saved: /content/my_data_ply_folder2/points3_view.ply
Saved: /content/my_data_ply_folder2/points4_view.ply
Saved: /content/my_data_ply_folder2/points5_view.ply
Saved: /content/my_data_ply_folder2/points6_view.ply
Saved: /content/my_data_ply_folder2/points7_view.ply
Saved: /content/my_data_ply_folder2/points8_view.ply
Saved: /content/my_data_ply_folder2/points9_view.ply
Saved: /content/my_data_ply_folder2/points10_view.ply
Saved: /content/my_data_ply_folder2/points11_view.ply
Saved: /content/my_data_ply_folder2/points12_view.ply
Saved: /content/my_data_ply_folder2/points13_view.ply
Saved: /content/my_data_ply_folder2/points14_view.ply
Saved: /content/my_data_ply_folder2/points15_view.ply
Saved: /content/my_data_ply_folder2/points16_view.ply
Saved: /content/my_data_ply_folder2/points17_view.ply
Saved: /content/my_data_ply_folder2/points18_view.ply
Saved: /content/my_data_ply_folder2/points19_view.ply
Saved: /content/my_data_ply_folder2/